## Imports

In [5]:
from time import strftime
from numpy.random import seed
seed(888)
import tensorflow as tf
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1 import set_random_seed
set_random_seed(404)

import os
import numpy as np

## Constants

In [6]:
X_TRAIN_PATH = './MNIST/digit_xtrain.csv'
X_TEST_PATH = './MNIST/digit_xtest.csv'
Y_TRAIN_PATH = './MNIST/digit_ytrain.csv'
Y_TEST_PATH = './MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_log'

IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUT = IMAGE_HEIGHT * IMAGE_WIDTH * CHANNELS

NUMBER_OF_CLASSES = 10
VALIDATION_SIZE = 10000

## Getting the data


In [7]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype = int)

CPU times: total: 141 ms
Wall time: 196 ms


In [8]:
y_train_all.shape

(60000,)

In [9]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype = int)

In [10]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype = int)

CPU times: total: 16.6 s
Wall time: 16.8 s


In [11]:
%%time

x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype = int)

CPU times: total: 2.77 s
Wall time: 2.79 s


## Data Exploration

In [12]:
x_train_all.shape

(60000, 784)

In [13]:
#0 means that the pixel is completely white whereas the value 255 indicates that the pixel is extremely dark
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [14]:
y_train_all.shape

(60000,)

In [15]:
x_test.shape

(10000, 784)

## Data Preprocessing

In [16]:
#Re-scaling
x_train_all, x_test = x_train_all/255.0, x_test/255.0

### Convert target values to one-hot encoding

In [17]:
y_train_all = np.eye(NUMBER_OF_CLASSES)[y_train_all]

In [18]:
y_train_all.shape

(60000, 10)

In [19]:
y_test = np.eye(NUMBER_OF_CLASSES)[y_test]

In [20]:
y_test.shape

(10000, 10)

## Create validation dataset from training dataset

In [21]:
x_validation = x_train_all[:VALIDATION_SIZE]
y_validation = y_train_all[:VALIDATION_SIZE]

In [22]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [23]:
x_train.shape

(50000, 784)

In [24]:
x_validation.shape

(10000, 784)

## Setting up TensorFlow graph

In [109]:
tf.compat.v1.disable_eager_execution()
X = tf.placeholder(tf.float32, shape = [None, TOTAL_INPUT], name='X')
Y = tf.placeholder(tf.float32, shape = [None, NUMBER_OF_CLASSES], name='labels')

### Setting up Neural Network

#### Hyperparameters

In [110]:
number_of_epochs = 5
learning_rate = 1e-4

hidden_layer_1 = 512
hidden_layer_2 = 64

In [111]:
with tf.name_scope('Hidden_layer_1'):
    #Creating the first layer
    initial_weight_1 = tf.truncated_normal(shape = [TOTAL_INPUT, hidden_layer_1], stddev=0.1, seed=42)
    weight_1 = tf.Variable(initial_value = initial_weight_1, name='W1')

    initial_bias_1 = tf.constant(value = 0.0, shape = [hidden_layer_1])
    bias_1 = tf.Variable(initial_value = initial_bias_1, name='B1')

    layer_1_input = tf.matmul(X, weight_1) + bias_1
    layer_1_output = tf.nn.relu(layer_1_input)

In [112]:
with tf.name_scope('Hidden_layer_2'):
    # Creating the second layer
    initial_weight_2 = tf.truncated_normal(shape = [hidden_layer_1, hidden_layer_2], stddev=0.1, seed=42)
    weight_2 = tf.Variable(initial_value = initial_weight_2, name='W2')

    initial_bias_2 = tf.constant(value = 0.0, shape = [hidden_layer_2])
    bias_2 = tf.Variable(initial_value = initial_bias_2, name='B2')

    layer_2_input = tf.matmul(layer_1_output, weight_2) + bias_2
    layer_2_output = tf.nn.relu( layer_2_input)

In [113]:
with tf.name_scope('output_layer'):
    # Creating the OUTPUT layer
    initial_weight_3 = tf.truncated_normal(shape =[hidden_layer_2,NUMBER_OF_CLASSES], stddev=0.1, seed=42)
    weight_3 = tf.Variable(initial_value = initial_weight_3, name='W3')

    initial_bias_3 = tf.constant(value = 0.0, shape = [NUMBER_OF_CLASSES])
    bias_3 = tf.Variable(initial_value = initial_bias_3, name='B3')

    layer_3_input = tf.matmul(layer_2_output, weight_3) + bias_3
    output = tf.nn.softmax( layer_3_input)

## Tensorboard Setup

In [114]:
#Folders for tensorboard
folder_name = f'Model 1 at {strftime("%H-%M")}'
directory = os.path.join(LOGGING_PATH, folder_name)
print(directory)
try:
    os.makedirs(directory) 
except OSError as exception:
    print(exception.strerror)
else:
    print('Sucessfully created directory')

tensorboard_mnist_digit_log\Model 1 at 22-19
Sucessfully created directory


## Loss, Optimisation and Metrics
### Defining Loss Function


In [115]:
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels =Y , logits=output))

### Defining Optimizer

In [116]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_step = optimizer.minimize(loss)

### Accuracy metric

In [117]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(output, axis=1) , tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [118]:
with tf.name_scope('performance'):
    tf.summary.scalar('cost', loss)
    tf.summary.scalar('accuracy', accuracy)

## Check input images in TensorBoard

In [119]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28 ,1])
    tf.summary.image('image_input', x_image, max_outputs=3)

## Run Session

In [120]:
session = tf.Session()

## Setup FileWriter and Merge Summaries

In [121]:
merged_summary = tf.summary.merge_all()
d_r = directory + '/train'
train_writer = tf.summary.FileWriter(d_r)
train_writer.add_graph(session.graph)

validation_writer = tf.summary.FileWriter(directory + '/validation')

## Initialize all varibales

In [122]:
#Initializing the variables 
init = tf.global_variables_initializer()
session.run(init)

In [123]:
bias_3.eval(session)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

## Making Batches

In [124]:
size_of_batch = 1000
num_examples = y_train.shape[0]
number_iterations = int(num_examples/ size_of_batch)

index_in_epoch = 0

In [125]:
def next_batch(batch_size, dataset, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return dataset[start: end], labels[start: end]

## Training loop

In [127]:
for epoch in range(number_of_epochs):
    # Training Datset
     for i in range(number_iterations):
            batch_x ,batch_y = next_batch(batch_size=size_of_batch, dataset = x_train, labels = y_train)
            
            feed_dictionary = {X: batch_x, Y: batch_y}
            
            session.run(train_step, feed_dict = feed_dictionary)
            
     s, batch_accuracy = session.run(fetches= [merged_summary, accuracy], feed_dict= feed_dictionary)
     train_writer.add_summary(s,epoch)
        
     print(f'Epoch {epoch} \t | Training Accuracy = {batch_accuracy}')
    
    # Validation Datset
     summary = session.run(fetches=merged_summary, feed_dict={X:x_validation, Y:y_validation})
     validation_writer.add_summary(summary, epoch)
    
print("Training complete")

InvalidArgumentError: Graph execution error:

Detected at node 'X' defined at (most recent call last):
    File "C:\Users\Mithun\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Mithun\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Mithun\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\Mithun\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\Mithun\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\Mithun\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\Mithun\anaconda3\lib\asyncio\base_events.py", line 601, in run_forever
      self._run_once()
    File "C:\Use

## Reset for the next Run

In [ ]:
train_writer.close()
validation_writer.close()
session.close()
tf.reset_default_graph()